<a href="https://colab.research.google.com/github/kashindra-mahato/NLP/blob/main/MultipleModelsFAQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## sentence transformers


In [ ]:
%%bash

nvidia-smi

Tue Mar 28 03:35:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 64.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-l5sam727/farm-haystack_43419819143340db98e298005e98a41b
  Resolved https://github.com/deepset-ai/haystack.git to commit c777302fb4052acded3302c76d1a78b47894f5e8
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Preparing meta

DEPRECATION: git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-l5sam727/farm-haystack_43419819143340db98e298005e98a41b


In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import EmbeddingRetriever, QuestionGenerator
import pandas as pd
from haystack.utils import fetch_archive_from_http
from haystack.pipelines import FAQPipeline
from haystack.pipelines import (
    QuestionGenerationPipeline,
    RetrieverQuestionGenerationPipeline,
    QuestionAnswerGenerationPipeline,
)
from haystack.schema import Answer
from haystack.utils import launch_es, print_questions

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).


In [ ]:
def predict(model_name, store_data, test_data):
  document_store = InMemoryDocumentStore()

  retriever = EmbeddingRetriever(
      document_store=document_store,
      embedding_model=model_name,
      use_gpu=True,
      scale_score=False,
  )

  doc_dir = "/"
  df = pd.read_excel(doc_dir+store_data)
  df.reset_index(drop=True)
  df = df[['question', 'answer']]
  df.columns = ['questions', 'answers']
  # Minimal cleaning
  df.fillna(value="", inplace=True)
  df["questions"] = df["questions"].apply(lambda x: x.strip())

  new_df = pd.DataFrame(df['questions'])
  new_df['questions'] = new_df['questions'].str.lower()
  new_df['questions'] = new_df['questions'].str.replace(r'[^\w\s]+', '')

  questions = list(new_df["questions"].values)
  new_df["embedding"] = retriever.embed_queries(queries=questions).tolist()
  new_df = new_df.rename(columns={"questions": "content"})

  docs_to_index = new_df.to_dict(orient="records")
  document_store.write_documents(docs_to_index)

  pipe = FAQPipeline(retriever=retriever)


  test_data = pd.read_excel(doc_dir + test_data)
  test_data.columns = ['questions']

  # Run any question and change top_k to see more or less answers
  prediction_list = [pipe.run(query=test_data['questions'][i], params={"Retriever": {"top_k": 1}}) for i in range(len(test_data['questions']))]

  query_list = [prediction_list[i]['query'] for i in range(len(prediction_list)-1)]
  context_list = [prediction_list[i]['answers'][0].context for i in range(len(prediction_list)-1)]
  score_list = [prediction_list[i]['answers'][0].score for i in range(len(prediction_list)-1)]
  result_df = pd.DataFrame(data=list(zip(query_list,context_list,score_list)), columns=['query', 'context', 'score'])
  print(result_df['score'].describe())
  result_df.to_excel(doc_dir+model_name+"_Queries.xlsx")
  return result_df

## sentence-transformers/all-MiniLM-L6-v2

In [ ]:
# predict("sentence-transformers/all-MiniLM-L6-v2",'data/.xlsx','test/combined_test.xlsx')

In [ ]:
# predict("sentence-transformers/all-MiniLM-L6-v2",'data/.xlsx','test/combined_test.xlsx')

In [ ]:
# predict("sentence-transformers/all-MiniLM-L6-v2",'data/.xlsx','test/combined_test.xlsx')

In [ ]:
# predict("sentence-transformers/all-MiniLM-L6-v2",'data/.xlsx','test/combined_test.xlsx')

In [ ]:
predict("sentence-transformers/all-MiniLM-L6-v2",'data/.xlsx','test/.xlsx')

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.nodes.retriever.dense:Init retriever using embeddings of model sentence-transformers/all-MiniLM-L6-v2
<ipython-input-22-d5979e558ab3>:22: FutureWarning: The default value of regex will change from True to False in a future version.
  new_df['questions'] = new_df['questions'].str.replace(r'[^\w\s]+', '')


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:haystack.document_stores.base:Duplicate Documents: Document with id '82bfb53e643f523a710f3ffd7fa5fcf8' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '65d62bafbb4d82101baa1a97b9b3051b' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '74a58c32035e42d43b48a65f1a0f8294' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id 'f53dd3245f802811cc4635ab6cc77677' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id 'cadf3987bf5426b608dad0affd526633' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '8e08d43bc2bf36c87d303628245d8432' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '1822e654ff90fd9fa133d9094c05bd6e' already exists in index 'document'

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

count    29.000000
mean      0.629309
std       0.144655
min       0.336702
25%       0.489576
50%       0.646469
75%       0.739820
max       0.854397
Name: score, dtype: float64


,query,context,score
0,Which university is best for applying uk?,which university will be best while applying for australia,0.679821
1,is there any chance of going to uk without doing ielts/pte?,are ielts and pte classes available,0.739820
2,how much score is needed in ielts to go to australia?,what is the ielts score to apply to australia parksa,0.767655
3,is ielts compulsory if we want to go to australia?,what is the ielts test for australia,0.730629
4,how can i be a canadian PR?,can i apply to canada if i am an academic,0.612409
5,what educational background is required to study PTE?,what are the classes of pte,0.684664
6,what is the cost of different universities in australia?,how much are the fees of universities and colleges in australia,0.806370
7,what is the cost of ielts prepparation?,what is the cost of ielts and pte,0.711254
8,what is the cost of pte preparation classes?,what is the price of ielts and pte classes now,0.739658
9,how can i apply for an offer letter?,parksa what to do to apply to canada,0.427160


## sentence-transformers/all-distilroberta-v1

In [ ]:
# predict("sentence-transformers/all-distilroberta-v1",'data/combined.xlsx','test/combined_test.xlsx')

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


INFO:haystack.nodes.retriever.dense:Init retriever using embeddings of model sentence-transformers/all-distilroberta-v1


<ipython-input-6-efc59a81cc3b>:21: FutureWarning: The default value of regex will change from True to False in a future version.
  new_df['questions'] = new_df['questions'].str.replace(r'[^\w\s]+', '')


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:haystack.document_stores.base:Duplicate Documents: Document with id 'd88d571f96f7067ff136872c3bafb871' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id 'cd6cdec1d32d8a9545edc02466234b44' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '5479dbcc09d1c0c255e6fc59f8b1598e' already exists in index 'document'


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

count    59.000000
mean      0.801608
std       0.099917
min       0.550139
25%       0.749501
50%       0.786029
75%       0.878595
max       0.988601
Name: score, dtype: float64


,query,context,score
0,What and where to study in Australia after my completion of 12 standard?,what and where to study in australia after my completion of 12 standard,0.951877
1,What are some recommended courses to study in Australia after finishing high...,what are the best australian courses,0.784027
2,Where can I find information on study options in Australia for students who ...,what and where to study in australia after my completion of 12 standard,0.790966
3,Which universities in Australia offer undergraduate programs for students wh...,what and where to study in australia after my completion of 12 standard,0.776379
4,"I want to study in Australia after completing my 12th standard, what are som...",what and where to study in australia after my completion of 12 standard,0.880712
5,Can you suggest some courses or majors to study in Australia after completin...,what and where to study in australia after my completion of 12 standard,0.863729
6,Is it possible to study in Australia without giving interview?,is it possible to study in australia without giving interview,0.955091
7,Can I study in Australia without being interviewed?,is it possible to study in australia without giving interview,0.880053
8,Is it mandatory to attend an interview to study in Australia?,is it possible to study in australia without giving interview,0.777896
9,Are there any options to avoid an interview while studying in Australia?,is it possible to study in australia without giving interview,0.780201


In [ ]:
predict("sentence-transformers/all-distilroberta-v1",'data/combined.xlsx','test/.xlsx')

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.nodes.retriever.dense:Init retriever using embeddings of model sentence-transformers/all-distilroberta-v1
<ipython-input-22-5dcbdfd2b0f6>:21: FutureWarning: The default value of regex will change from True to False in a future version.
  new_df['questions'] = new_df['questions'].str.replace(r'[^\w\s]+', '')


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:haystack.document_stores.base:Duplicate Documents: Document with id 'd88d571f96f7067ff136872c3bafb871' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id 'cd6cdec1d32d8a9545edc02466234b44' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '5479dbcc09d1c0c255e6fc59f8b1598e' already exists in index 'document'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

count    29.000000
mean      0.654199
std       0.120771
min       0.395274
25%       0.563806
50%       0.680530
75%       0.717440
max       0.884066
Name: score, dtype: float64


,query,context,score
0,Which university is best for applying uk?,if i cant apply inside london where i can apply,0.511668
1,is there any chance of going to uk without doing ielts/pte?,do you provide a student visa without ielts or pte,0.713811
2,how much score is needed in ielts to go to australia?,is ieltspte a requirement for studying in australia,0.717440
3,is ielts compulsory if we want to go to australia?,is ieltspte a requirement for studying in australia,0.808904
4,how can i be a canadian PR?,is it easy to get a job in canada after pg diploma,0.520511
5,what educational background is required to study PTE?,do you have pte ielts class,0.549418
6,what is the cost of different universities in australia?,how much will it cost in to study in australia for graduate program,0.715660
7,what is the cost of ielts prepparation?,what is the cost of the ielts class,0.675967
8,what is the cost of pte preparation classes?,what is the cost of the ielts class,0.563806
9,how can i apply for an offer letter?,after gaining an acceptance letter what do i do next,0.535904


## sentence-transformers/multi-qa-distilbert-cos-v1

In [ ]:
# predict("sentence-transformers/multi-qa-distilbert-cos-v1",'data/combined.xlsx','test/combined_test.xlsx')

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


INFO:haystack.nodes.retriever.dense:Init retriever using embeddings of model sentence-transformers/multi-qa-distilbert-cos-v1


<ipython-input-6-efc59a81cc3b>:21: FutureWarning: The default value of regex will change from True to False in a future version.
  new_df['questions'] = new_df['questions'].str.replace(r'[^\w\s]+', '')


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:haystack.document_stores.base:Duplicate Documents: Document with id 'd88d571f96f7067ff136872c3bafb871' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id 'cd6cdec1d32d8a9545edc02466234b44' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '5479dbcc09d1c0c255e6fc59f8b1598e' already exists in index 'document'


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

count    59.000000
mean      0.825382
std       0.112620
min       0.528901
25%       0.763393
50%       0.829754
75%       0.897130
max       0.996257
Name: score, dtype: float64


,query,context,score
0,What and where to study in Australia after my completion of 12 standard?,what and where to study in australia after my completion of 12 standard,0.992415
1,What are some recommended courses to study in Australia after finishing high...,what are the best australian courses,0.743492
2,Where can I find information on study options in Australia for students who ...,what and where to study in australia after my completion of 12 standard,0.829754
3,Which universities in Australia offer undergraduate programs for students wh...,what and where to study in australia after my completion of 12 standard,0.790267
4,"I want to study in Australia after completing my 12th standard, what are som...",what and where to study in australia after my completion of 12 standard,0.897250
5,Can you suggest some courses or majors to study in Australia after completin...,what and where to study in australia after my completion of 12 standard,0.848572
6,Is it possible to study in Australia without giving interview?,is it possible to study in australia without giving interview,0.996257
7,Can I study in Australia without being interviewed?,is it possible to study in australia without giving interview,0.954589
8,Is it mandatory to attend an interview to study in Australia?,is it possible to study in australia without giving interview,0.828118
9,Are there any options to avoid an interview while studying in Australia?,is it possible to study in australia without giving interview,0.874642


In [ ]:
predict("sentence-transformers/multi-qa-distilbert-cos-v1",'data/combined.xlsx','test/.xlsx')

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.nodes.retriever.dense:Init retriever using embeddings of model sentence-transformers/multi-qa-distilbert-cos-v1
<ipython-input-22-5dcbdfd2b0f6>:21: FutureWarning: The default value of regex will change from True to False in a future version.
  new_df['questions'] = new_df['questions'].str.replace(r'[^\w\s]+', '')


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:haystack.document_stores.base:Duplicate Documents: Document with id 'd88d571f96f7067ff136872c3bafb871' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id 'cd6cdec1d32d8a9545edc02466234b44' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '5479dbcc09d1c0c255e6fc59f8b1598e' already exists in index 'document'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

count    29.000000
mean      0.648719
std       0.116360
min       0.461787
25%       0.576946
50%       0.662928
75%       0.721687
max       0.879694
Name: score, dtype: float64


,query,context,score
0,Which university is best for applying uk?,is there any university that accepts without ielts,0.591104
1,is there any chance of going to uk without doing ielts/pte?,do you provide a student visa without ielts or pte,0.716262
2,how much score is needed in ielts to go to australia?,is ieltspte a requirement for studying in australia,0.755139
3,is ielts compulsory if we want to go to australia?,is ieltspte a requirement for studying in australia,0.879694
4,how can i be a canadian PR?,what are the requirements for admission into canadian collegesuniversities,0.579073
5,what educational background is required to study PTE?,do you have pte ielts class,0.576946
6,what is the cost of different universities in australia?,how much will it cost to studying in australia for bachelor program,0.684523
7,what is the cost of ielts prepparation?,what is the cost of the ielts class,0.794569
8,what is the cost of pte preparation classes?,what is the cost of the ielts class,0.498674
9,how can i apply for an offer letter?,how fast can we get the offer letter,0.593285


In [ ]:
# Recommended: Start Elasticsearch using Docker via the Haystack utility function
from haystack.utils import launch_es

launch_es()

In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2


In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import BM25Retriever
import os
import time

## BM25Retriever

In [ ]:
# Wait 30 seconds only to be sure Elasticsearch is ready before continuing
time.sleep(30)

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

# Initialize DocumentStore and index documents
document_store = ElasticsearchDocumentStore(host=host)

retriever = BM25Retriever(document_store=document_store)

doc_dir = "/"
df = pd.read_excel(f"{doc_dir}/.xlsx")
df.columns = ['questions', 'answers']
# Minimal cleaning
df.fillna(value="", inplace=True)
df["questions"] = df["questions"].apply(lambda x: x.strip())

new_df = pd.DataFrame(df['questions'])
new_df['questions'] = new_df['questions'].str.lower()
new_df['questions'] = new_df['questions'].str.replace(r'[^\w\s]+', '')

questions = list(new_df["questions"].values)
# new_df["embedding"] = retriever.embed_queries(queries=questions).tolist()
new_df = new_df.rename(columns={"questions": "content"})

docs_to_index = new_df.to_dict(orient="records")
document_store.write_documents(docs_to_index)

pipe = FAQPipeline(retriever=retriever)


test_data = pd.read_excel(doc_dir + '_test.xlsx')
test_data.columns = ['questions']

# Run any question and change top_k to see more or less answers
prediction_list = [pipe.run(query=test_data['questions'][i], params={"Retriever": {"top_k": 1}}) for i in range(len(test_data['questions']))]

query_list = [prediction_list[i]['query'] for i in range(len(prediction_list)-1)]
context_list = [prediction_list[i]['answers'][0].context for i in range(len(prediction_list)-1)]
score_list = [prediction_list[i]['answers'][0].score for i in range(len(prediction_list)-1)]
result_df = pd.DataFrame(data=list(zip(query_list,context_list,score_list)), columns=['query', 'context', 'score'])
print(result_df['score'].describe())
# result_df.to_excel(doc_dir+model_name+".xlsx")

<ipython-input-20-19ad4f7c9a0b>:21: FutureWarning: The default value of regex will change from True to False in a future version.
  new_df['questions'] = new_df['questions'].str.replace(r'[^\w\s]+', '')


count    28.000000
mean      0.765191
std       0.070993
min       0.592001
25%       0.732901
50%       0.764620
75%       0.813421
max       0.907729
Name: score, dtype: float64


In [ ]:
result_df

## Question generation, Retrieval Question, and Question answering pipeline

In [ ]:
import pandas as pd

In [ ]:
def predict(store_data):
  document_store = InMemoryDocumentStore()

  doc_dir = "/"
  df = pd.read_excel(doc_dir+store_data)
  df.reset_index(drop=True)
  new_df = df[['answer']]
  new_df.columns = ['content']
  new_df.head()
  # Minimal cleaning
  new_df.fillna(value="", inplace=True)
  new_df["content"] = new_df["content"].apply(lambda x: x.strip())
  new_df['content'] = new_df['content'].str.lower()
  new_df['content'] = new_df['content'].str.replace(r'[^\w\s]+', '')


  docs_to_index = new_df.to_dict(orient="records")
  document_store.write_documents(docs_to_index)

  question_generator = QuestionGenerator()
  question_generation_pipeline = QuestionGenerationPipeline(question_generator)

  for idx, document in enumerate(document_store):

    print(f"\n * Generating questions for document {idx}: {document.content}...\n")
    result = question_generation_pipeline.run(documents=[document])
    print_questions(result)


In [ ]:
predict('/data/.xlsx')

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
<ipython-input-141-ffe3951734dd>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.fillna(value="", inplace=True)
<ipython-input-141-ffe3951734dd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["content"] = new_df["content"].apply(lambda x: x.strip())
<ipython-input-141-ffe3951734dd>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

KeyboardInterrupt: ignored

# Extractive pipeline

In [ ]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.8/677.8 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 88.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.


In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)


In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2


In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch


In [ ]:
import time
time.sleep(30)

In [ ]:
from haystack.utils import launch_es
launch_es()

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).


In [ ]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

document_store = ElasticsearchDocumentStore(
    host=host,
    username="",
    password="",
    index="document",
    embedding_dim=768
)

## import and indexing text


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
doc_dir = '/'

In [ ]:
from haystack import Pipeline
from haystack.nodes import TextConverter, PreProcessor

indexing_pipeline = Pipeline()
text_converter = TextConverter()
preprocessor = PreProcessor(
    clean_whitespace=True,
    clean_header_footer=True,
    clean_empty_lines=False,
    split_by="passage",
    split_length=1,
    # split_overlap=20,
    split_respect_sentence_boundary=False,
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import os

indexing_pipeline.add_node(component=text_converter, name="TextConverter", inputs=["File"])
indexing_pipeline.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"])
indexing_pipeline.add_node(component=document_store, name="DocumentStore", inputs=["PreProcessor"])


In [ ]:
files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline.run_batch(file_paths=files_to_index)


INFO:haystack.pipelines.base:It seems that an indexing Pipeline is run, so using the nodes' run method instead of run_batch.


Converting files:   0%|          | 0/12 [00:00<?, ?it/s]

Preprocessing:   0%|          | 0/12 [00:00<?, ?docs/s]

{'documents': [<Document: {'content': 'pre-departure and on arrival support\nVisiting any foreign country for studying, working and living can be tedious and complex in the beginning. Likewise, students who are getting ready to fly may get stressed. We at Aus Studies, understand students such circumstances and always keep ourselves alert to provide the necessary assistance it requires.\nTo every visa granted student, we counsel individuals and often provide a group orientation about air ticketing, travel guide, accommodation, air port pick etc. To ease your initial days on foreign land, we organize travel, stay and culture training as you might get it so important.\nThe main objectives of such training are to make the students feel comfortable living in a foreign country and avoid/minimize culture shock. We will brief you on what to expect while living and studying abroad and plan for challenges you might face during your stay.', 'content_type': 'text', 'score': None, 'meta': {'_split_

In [ ]:
import pandas as pd

In [ ]:
# load the query
query_df = pd.read_excel('/.xlsx')

In [ ]:
query_df.head()

# simple retriever and reader pipeline

## initializing the retriever

In [ ]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

## initializing the reader

In [ ]:
from haystack.nodes import FARMReader
from haystack.nodes import Seq2SeqGenerator
# from haystack.nodes import TransformersSummarizer
# reader = TransformersSummarizer(model_name_or_path="snrspeaks/t5-one-line-summary")
# reader = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)


## creating a reader and retrieval pipeline

In [ ]:
from haystack import Pipeline

querying_pipeline = Pipeline()
querying_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
querying_pipeline.add_node(component=reader, name="Reader", inputs=["Retriever"])


In [ ]:
doc_dir

In [ ]:
import pandas as pd

In [ ]:

def query(test_data, pipe):
  prediction_list = [pipe.run(query=test_data['questions'][i], params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 1}}) for i in range(len(test_data['questions']))]

  query_list = [prediction_list[i]['query'] for i in range(len(prediction_list)-1)]
  context_list = [prediction_list[i]['documents'][0].content for i in range(len(prediction_list)-1)]
  answer_list = [prediction_list[i]['answers'][0].answer for i in range(len(prediction_list)-1)]
  score_list = [prediction_list[i]['documents'][0].score for i in range(len(prediction_list)-1)]
  result_df = pd.DataFrame(data=list(zip(query_list,context_list,answer_list,score_list)), columns=['query', 'context','answer', 'score'])
  print(result_df['score'].describe())
  return result_df


In [ ]:
predicted = query(query_df, querying_pipeline)

In [ ]:
predicted.to_excel('/.xlsx')

# keyword, interrogative and statement queries

In [ ]:

# Here we create the keyword vs question/statement query classifier

from haystack.nodes import SklearnQueryClassifier
keyword_classifier = SklearnQueryClassifier()

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DummyClassifier from version 1.2.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.2.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator GradientBoostingClassifier from version 1.2.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your

In [ ]:


#Examples for Query Classifier
queries = [
    "",  # Keyword Query
    "",  # Interrogative Query
    "",  # Statement Query
]

In [ ]:

import pandas as pd

k_vs_qs_results = {"Query": [], "Output Branch": [], "Class": []}

for query in queries:
    result = keyword_classifier.run(query=query)
    print(result)
    k_vs_qs_results["Query"].append(query)
    k_vs_qs_results["Output Branch"].append(result[1])
    k_vs_qs_results["Class"].append("Question/Statement" if result[1] == "output_1" else "Keyword")

pd.DataFrame.from_dict(k_vs_qs_results)

In [ ]:
# Here we create the question vs statement query classifier
model_url = (
    "https://ext-models-haystack.s3.eu-central-1.amazonaws.com/gradboost_query_classifier_statements/model.pickle"
)
vectorizer_url = (
    "https://ext-models-haystack.s3.eu-central-1.amazonaws.com/gradboost_query_classifier_statements/vectorizer.pickle"
)

question_classifier = SklearnQueryClassifier(model_name_or_path=model_url, vectorizer_name_or_path=vectorizer_url)

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DummyClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator GradientBoostingClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at y

In [ ]:
queries = [
    "",  # Interrogative Query
    "",  # Statement Query
]

q_vs_s_results = {"Query": [], "Output Branch": [], "Class": []}

for query in queries:
    result = question_classifier.run(query=query)
    q_vs_s_results["Query"].append(query)
    q_vs_s_results["Output Branch"].append(result[1])
    q_vs_s_results["Class"].append("Question" if result[1] == "output_1" else "Statement")

pd.DataFrame.from_dict(q_vs_s_results)

In [ ]:
from haystack.nodes import BM25Retriever, DensePassageRetriever, FARMReader, SentenceTransformersRanker


# Initialize sparse retriever for keyword queries
bm25_retriever = BM25Retriever(document_store=document_store)

# Initialize dense retriever for question/statement queries
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    use_gpu=True,
    embed_title=True,
)

#update the embedding for retriever
document_store.update_embeddings(retriever)


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
INFO:haystack.modeling.model.language_model:Auto-detected model language: english


Updating embeddings:   0%|          | 0/123 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/128 [00:00<?, ? Docs/s]

In [ ]:
from haystack.nodes import FARMReader
from haystack.nodes import TransformersSummarizer, Seq2SeqGenerator

# reader = TransformersSummarizer(model_name_or_path="snrspeaks/t5-one-line-summary")
# reader = TransformersSummarizer(model_name_or_path='prithivida/parrot_paraphraser_on_T5')
reader = TransformersSummarizer(model_name_or_path='knkarthick/MEETING-SUMMARY-BART-LARGE-XSUM-SAMSUM-DIALOGSUM-AMI')
farmreader = FARMReader(model_name_or_path="deepset/roberta-base-squad2")
# generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")
# ranker = SentenceTransformersRanker(model_name_or_path="cross-encoder/ms-marco-MiniLM-L-12-v2")

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [ ]:

querying_pipeline = Pipeline()
querying_pipeline.add_node(component=bm25_retriever, name="BM25Retriever", inputs=["Query"])
querying_pipeline.add_node(component=farmreader, name="Reader", inputs=["BM25Retriever"])

In [ ]:
# from haystack.pipelines import Pipeline


# # Here we build the pipeline
# qa_pipeline = Pipeline()
# qa_pipeline.add_node(component=question_classifier, name="QueryClassifier", inputs=["Query"])
# # branch to DPR for interrogative queries (output_1)
# qa_pipeline.add_node(component=retriever, name="DPR", inputs=["QueryClassifier.output_1"])
# # branch to BM25Retriever for keyword based queries (output_2)
# qa_pipeline.add_node(component=bm25_retriever, name="BM25Retriever", inputs=["QueryClassifier.output_2"])
# qa_pipeline.add_node(component=farmreader, name="QAReader", inputs=["DPR","BM25Retriever"])
# # qa_pipeline.add_node(component=generator, name="Generator", inputs=["Reader"])
# # qa_pipeline.add_node(component=reader, name="QAReader", inputs=["DPR",'BM25Retriever'])
# # qa_pipeline.add_node(component=ranker, name="Ranker", inputs=["BM25Retriever"])


In [ ]:
if qa_pipeline.components['QueryClassifier'].run(query='')[1] == 'output_1':
  print('True')

True


In [ ]:
import numpy as np

In [ ]:
from numpy.core.multiarray import empty
def query(test_data, pipe):
  prediction_list = []
  query_list = []
  context_list = []
  answer_list = []
  score_list = []
  for i in range(len(test_data['questions'])):
    if pipe.components['QueryClassifier'].run(query=test_data['questions'][i])[1] == 'output_1':
      prediction_list.append(pipe.run(query=test_data['questions'][i], params={"DPR": {"top_k": 5}}))
      query_list.append(prediction_list[i]['query'])
      # print(prediction_list[i]['answers'][0].context,"\n",pipe.components['QueryClassifier'].run(query=test_data['questions'][i])[1])
      context_list.append(prediction_list[i]['documents'][0].content)
      print(prediction_list[i]['query'])
      if prediction_list[i]['answers']:
        answer_list.append(prediction_list[i]['answers'][0].answer)
      else:
        answer_list.append(np.nan)
      score_list.append(prediction_list[i]['documents'][0].score)

    else:
      prediction_list.append(pipe.run(query=test_data['questions'][i], params={"BM25Retriever": {"top_k": 5}}))
      query_list.append(test_data['questions'][i])
      context_list.append(prediction_list[i]['documents'][0].content)
      answer_list.append(np.nan)
      score_list.append(prediction_list[i]['documents'][0].score)

  result_df = pd.DataFrame(data=list(zip(query_list,context_list,answer_list,score_list)), columns=['query', 'context','answer', 'score'])
  print(result_df['score'].describe())
  return result_df


In [ ]:
# result = query(query_df, qa_pipeline)

In [ ]:
# result.to_excel('/content/drive/MyDrive/Colab Notebooks/Palmmind/consultancy/results/questionClassified_roberta_base.xlsx')

In [ ]:

from haystack.utils import print_answers
from time import time

# Useful for framing headers
equal_line = "=" * 30

# Run only the dense retriever on the full sentence query
t1 = time()
res_1 = querying_pipeline.run(query='',
params={
        "BM25Retriever": {"top_k": 10},
        # "QAReader": {"top_k": 5}
    })
t2 = time()
print("required time: ", t2-t1)
print(f"\n\n{equal_line}\nQUESTION QUERY RESULTS\n{equal_line}")
# print_answers(res_1, details="minimum")
print(res_1['query'])
print(res_1['answers'][0].answer)
# print(res_1)
# print(res_1['documents'][0].meta['summary'])
print("\n\n")



In [ ]:
res_3 = reader.summarizer("question:"+" "+ res_1['query']+" " +"answer:"+" "+res_1['answers'][0].answer)

Your max_length is set to 62, but you input_length is only 42. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=21)


In [ ]:
res_3

In [ ]:
# Run only the sparse retriever on a keyword based query
res_2 = qa_pipeline.run(query="",
                        params={
        "BM25Retriever": {"top_k": 1},
        # "Ranker": {"top_k": 5}
    })
print(f"\n\n{equal_line}\nKEYWORD QUERY RESULTS\n{equal_line}")
print(res_2)
# print_answers(res_2, details="minimum")

In [ ]:
for doc in res_2['documents']:
  print(doc.content)
  print(doc.score)


In [ ]:
from haystack.pipelines import QuestionGenerationPipeline
from haystack.nodes import QuestionGenerator

In [ ]:
question_generator = QuestionGenerator()
question_generation_pipeline = QuestionGenerationPipeline(question_generator)

temp_dir = {}
for idx, document in enumerate(document_store):
  # print(f"\n * Generating questions for document {idx}: {document.content}...\n")
  result = question_generation_pipeline.run(documents=[document])
  temp_dir.update({idx:result['generated_questions'][0]['questions']})

NameError: ignored

In [ ]:
temp_dir

In [ ]:
temp_list = []
for key, value in temp_dir.items():
  for i in range(len(value)):
    temp_list.append(value[i])

In [ ]:
temp_list

In [ ]:
import pandas as pd

In [ ]:
# temp_df = pd.DataFrame(temp_list, columns=['questions'])
temp_df = pd.read_excel(doc_dir+"/generated_questions.xlsx")

In [ ]:
temp_df.dropna(inplace=True)

In [ ]:
temp_df.head()

In [ ]:
temp_df['questions'].to_excel(doc_dir+"/generated_questions.xlsx")

## Generative pipeline

In [ ]:
%%bash

nvidia-smi


Wed Mar 29 10:42:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-o8hiqc3q/farm-haystack_ab5e8d3e3d4f4befb8dbb72d26af8e25
  Resolved https://github.com/deepset-ai/haystack.git to commit 16bd7d0625e3f8f7b87e0f9d1a455f8154ce4dff
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 

DEPRECATION: git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-o8hiqc3q/farm-haystack_ab5e8d3e3d4f4befb8dbb72d26af8e25


In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)


In [ ]:
import pandas as pd

from haystack.utils import fetch_archive_from_http


# Download sample
doc_dir = "data/tutorial7/"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/small_generator_dataset.csv.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# Create dataframe with columns "title" and "text"
df = pd.read_csv(f"{doc_dir}/small_generator_dataset.csv", sep=",")
# Minimal cleaning
df.fillna(value="", inplace=True)

print(df.head())


INFO:haystack.utils.import_utils:Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/small_generator_dataset.csv.zip to 'data/tutorial7/'


               title  \
0  "Albert Einstein"   
1  "Albert Einstein"   
2  "Albert Einstein"   
3  "Albert Einstein"   
4     "Alfred Nobel"   

                                                                              text  
0  to Einstein in 1922. Footnotes Citations Albert Einstein Albert Einstein (; ...  
1  Albert Einstein Albert Einstein (; ; 14 March 1879 – 18 April 1955) was a Ge...  
2  observations were published in the international media, making Einstein worl...  
3  model for depictions of mad scientists and absent-minded professors; his exp...  
4  was adopted as the standard technology for mining in the "Age of Engineering...  


In [ ]:
from haystack import Document


# Use data to initialize Document objects
titles = list(df["title"].values)
texts = list(df["text"].values)
documents = []
for title, text in zip(titles, texts):
    documents.append(Document(content=text, meta={"name": title or ""}))


In [ ]:
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import RAGenerator, DensePassageRetriever


# Initialize FAISS document store.
# Set `return_embedding` to `True`, so generator doesn't have to perform re-embedding
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", return_embedding=True)

# Initialize DPR Retriever to encode documents, encode question and query documents
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    use_gpu=True,
    embed_title=True,
)

# Initialize RAG Generator
generator = RAGenerator(
    model_name_or_path="facebook/rag-token-nq",
    use_gpu=True,
    top_k=1,
    max_length=200,
    min_length=2,
    embed_title=True,
    num_beams=2,
)


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
/usr/local/lib/python3.9/dist-packages/transformers/models/bart/configuration_bart.py:179: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fro

In [ ]:
# Delete existing documents in documents store
document_store.delete_documents()

# Write documents to document store
document_store.write_documents(documents)

# Add documents embeddings to index
document_store.update_embeddings(retriever=retriever)


Writing Documents:   0%|          | 0/75 [00:00<?, ?it/s]

INFO:haystack.document_stores.faiss:Updating embeddings for 68 docs...


Updating Embedding:   0%|          | 0/68 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/80 [00:00<?, ? Docs/s]

In [ ]:
QUESTIONS = [
    "who got the first nobel prize in physics",
    "when is the next deadpool movie being released",
    "which mode is used for short wave broadcast service",
    "who is the owner of reading football club",
    "when is the next scandal episode coming out",
    "when is the last time the philadelphia won the superbowl",
    "what is the most current adobe flash player version",
    "how many episodes are there in dragon ball z",
    "what is the first step in the evolution of the eye",
    "where is gall bladder situated in human body",
    "what is the main mineral in lithium batteries",
    "who is the president of usa right now",
    "where do the greasers live in the outsiders",
    "panda is a national animal of which country",
    "what is the name of manchester united stadium",
]


In [ ]:
# Or alternatively use the Pipeline class
from haystack.pipelines import GenerativeQAPipeline
from haystack.utils import print_answers

pipe = GenerativeQAPipeline(generator=generator, retriever=retriever)
for question in QUESTIONS:
    res = pipe.run(query=question, params={"Generator": {"top_k": 1}, "Retriever": {"top_k": 5}})
    print_answers(res, details="minimum")


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:2734: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


'Query: who got the first nobel prize in physics'
'Answers:'
[{'answer': ' albert einstein'}]
'Query: when is the next deadpool movie being released'
'Answers:'
[{'answer': ' september 22, 2017'}]
'Query: which mode is used for short wave broadcast service'
'Answers:'
[{'answer': ' amplitude modulation'}]
'Query: who is the owner of reading football club'
'Answers:'
[{'answer': ' stefan persson'}]
'Query: when is the next scandal episode coming out'
'Answers:'
[{'answer': ' april 20, 2018'}]
'Query: when is the last time the philadelphia won the superbowl'
'Answers:'
[{'answer': ' the 1970s'}]
'Query: what is the most current adobe flash player version'
'Answers:'
[{'answer': ' 7.1. 2'}]
'Query: how many episodes are there in dragon ball z'
'Answers:'
[{'answer': ' 13'}]
'Query: what is the first step in the evolution of the eye'
'Answers:'
[{'answer': ' step by step'}]
'Query: where is gall bladder situated in human body'
'Answers:'
[{'answer': ' stomach'}]
'Query: what is the main mi

# Prompt node

In [ ]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)


In [ ]:
from haystack.pipelines import Pipeline
from haystack.nodes import  PromptNode, PromptTemplate, AnswerParser
from haystack.schema import Document

# Let's create a custom LFQA prompt using PromptTemplate
lfqa_prompt = PromptTemplate(name="lfqa",
                             prompt_text="""Synthesize a comprehensive answer from the following topk most relevant paragraphs and the given question.
                             Provide a clear and concise response that summarizes the key points and information presented in the paragraphs.
                             Your answer should be in your own words and be no longer than 50 words.
                             \n\n Paragraphs: {join(documents)} \n\n Question: {query} \n\n Answer:""",
                             output_parser=AnswerParser(),)

# These docs could also come from a retriever
# Here we explicitly specify them to avoid the setup steps for Retriever and DocumentStore
doc_1 = "Contrails are a manmade type of cirrus cloud formed when water vapor from the exhaust of a jet engine condenses on particles, which come from either the surrounding air or the exhaust itself, and freezes, leaving behind a visible trail. The exhaust can also trigger the formation of cirrus by providing ice nuclei when there is an insufficient naturally-occurring supply in the atmosphere. One of the environmental impacts of aviation is that persistent contrails can form into large mats of cirrus, and increased air traffic has been implicated as one possible cause of the increasing frequency and amount of cirrus in Earth's atmosphere."
doc_2 = "Because the aviation industry is especially sensitive to the weather, accurate weather forecasting is essential. Fog or exceptionally low ceilings can prevent many aircraft from landing and taking off. Turbulence and icing are also significant in-flight hazards. Thunderstorms are a problem for all aircraft because of severe turbulence due to their updrafts and outflow boundaries, icing due to the heavy precipitation, as well as large hail, strong winds, and lightning, all of which can cause severe damage to an aircraft in flight. Volcanic ash is also a significant problem for aviation, as aircraft can lose engine power within ash clouds. On a day-to-day basis airliners are routed to take advantage of the jet stream tailwind to improve fuel efficiency. Aircrews are briefed prior to takeoff on the conditions to expect en route and at their destination. Additionally, airports often change which runway is being used to take advantage of a headwind. This reduces the distance required for takeoff, and eliminates potential crosswinds."


# Let's initiate the PromptNode
node = PromptNode("google/flan-t5-base", default_prompt_template=lfqa_prompt)

pipe = Pipeline()
pipe.add_node(component=node, name="prompt_node", inputs=["Query"])

output = pipe.run(query="Why do airplanes leave contrails in the sky?", documents=[Document(doc_1), Document(doc_2)])
[a.answer for a in output["answers"]]

# Here's the answer:
["Contrails are manmade clouds formed when water vapor from the exhaust of a jet engine condenses on particles, which come from either the surrounding air or the exhaust itself, and freezes, creating a visible trail. Increased air traffic has been linked to the greater frequency and amount of these cirrus clouds in Earth's atmosphere."]

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


["Contrails are manmade clouds formed when water vapor from the exhaust of a jet engine condenses on particles, which come from either the surrounding air or the exhaust itself, and freezes, creating a visible trail. Increased air traffic has been linked to the greater frequency and amount of these cirrus clouds in Earth's atmosphere."]

In [ ]:
print(output)

{'answers': [<Answer {'answer': 'jet stream tailwind.', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['6a371f0bbb37c291befaaaf4704dc694', '2a2f7c49e1bec7864dd4bb447d5d0bfa'], 'meta': {'prompt': "Synthesize a comprehensive answer from the following topk most relevant paragraphs and the given question. Provide a clear and concise response that summarizes the key points and information presented in the paragraphs. Your answer should be in your own words and be no longer than 50 words. Paragraphs: Contrails are a manmade type of cirrus cloud formed when water vapor from the exhaust of a jet engine condenses on particles, which come from either the surrounding air or the exhaust itself, and freezes, leaving behind a visible trail. The exhaust can also trigger the formation of cirrus by providing ice nuclei when there is an insufficient naturally-occurring supply in the atmosphere. One of the environmental impa

In [ ]:
from haystack import Pipeline
from haystack.nodes.prompt import PromptNode, PromptModel
# You'd also need to import a Retriever and a DocumentStore,
# we're skipping this in this example

top_k = 10
query = "Who is Paul Atreides' father?"

prompt_model = PromptModel()
node = PromptNode(prompt_model, default_prompt_template="question-generation", output_variable="query")
node2 = PromptNode(prompt_model, default_prompt_template="question-answering-per-document")

# You'd also need to initialize a Retriever with a DocumentStore
# We're skipping this step in this example to simplify it
pipe = Pipeline()
pipe.add_node(component=retriever, name="retriever", inputs=["Query"])
pipe.add_node(component=node, name="prompt_node", inputs=["retriever"])
pipe.add_node(component=node2, name="prompt_node_2", inputs=["prompt_node"])
output = pipe.run(query=query, params={"retriever": {"top_k": 10}})

dict(zip(output["query"], output["answers"]))